In [116]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm

In [117]:
# pre-process ny state mobility data

# load nyc data
ny_mobility = pd.read_csv('/Users/luchen/Documents/MSUA/Capstone/datasets/Google Mobility Data/ny_mobility.csv', na_values='')

# merge nyc data
ny_mobility = ny_mobility.drop(['month', 'day'], axis=1)

print("Data shape:", ny_mobility.shape)
print(ny_mobility.head())

Data shape: (60126, 16)
  country_region_code country_region sub_region_1 sub_region_2  metro_area  \
0                  US  United States     New York          NaN         NaN   
1                  US  United States     New York          NaN         NaN   
2                  US  United States     New York          NaN         NaN   
3                  US  United States     New York          NaN         NaN   
4                  US  United States     New York          NaN         NaN   

  iso_3166_2_code  census_fips_code                     place_id        date  \
0           US-NY               NaN  ChIJqaUj8fBLzEwRZ5UY3sHGz90  2020-02-15   
1           US-NY               NaN  ChIJqaUj8fBLzEwRZ5UY3sHGz90  2020-02-16   
2           US-NY               NaN  ChIJqaUj8fBLzEwRZ5UY3sHGz90  2020-02-17   
3           US-NY               NaN  ChIJqaUj8fBLzEwRZ5UY3sHGz90  2020-02-18   
4           US-NY               NaN  ChIJqaUj8fBLzEwRZ5UY3sHGz90  2020-02-19   

   retail_and_recreation_p

In [118]:
# filter ny state acs data
acs = pd.read_csv('/Users/luchen/Documents/MSUA/Capstone/datasets/Census/acs_var_filtered.csv', na_values='')
ny_acs = acs.loc[(acs['Geo_STUSAB'] == 'ny') ].copy()

# merge two dfs
ny_acs['CountyName'] = ny_acs['Geo_QName'].str.split(',').str[0].str.strip()

# Perform the join
ny_merge = pd.merge(ny_acs, ny_mobility, left_on='CountyName', right_on='sub_region_2')
# print(ny_merge.head())

In [119]:
# List of socioeconomic variables you want to consider
# Select columns to keep
columns_to_keep = ['CountyName', 'date', 
                 'retail_and_recreation_percent_change_from_baseline', 
                 'grocery_and_pharmacy_percent_change_from_baseline', 
                 'parks_percent_change_from_baseline', 
                 'transit_stations_percent_change_from_baseline', 
                 'workplaces_percent_change_from_baseline', 
                 'residential_percent_change_from_baseline'] + list(ny_merge.columns[5:-16])

# Create subset dataframe
ny_model = ny_merge.loc[:, columns_to_keep].copy()
ny_model = ny_model.iloc[:, :-1]
# ny_model = ny_model.dropna()

# Print the subset dataframe
print(ny_model.head())
print(ny_model.shape)
# (59152, 45)

      CountyName        date  \
0  Albany County  2020-02-15   
1  Albany County  2020-02-16   
2  Albany County  2020-02-17   
3  Albany County  2020-02-18   
4  Albany County  2020-02-19   

   retail_and_recreation_percent_change_from_baseline  \
0                                                8.0    
1                                                7.0    
2                                               11.0    
3                                               -7.0    
4                                                4.0    

   grocery_and_pharmacy_percent_change_from_baseline  \
0                                               -4.0   
1                                               -6.0   
2                                                2.0   
3                                               -7.0   
4                                               -3.0   

   parks_percent_change_from_baseline  \
0                                45.0   
1                                 8.0   
2   

In [121]:
# combine occupations
ny_model['PCT_+4_ocp'] = ny_model['PCT_SE_B17008_002'] + ny_model['PCT_SE_B17008_004'] 
ny_model['PCT_+2_ocp'] = ny_model['PCT_SE_B17008_003']
ny_model['PCT_+1_ocp'] = ny_model['PCT_SE_B17008_011'] + ny_model['PCT_SE_B17008_008']
ny_model['PCT_-1_ocp'] = ny_model['PCT_SE_B17008_007'] + ny_model['PCT_SE_B17008_002'] + ny_model['PCT_SE_B17008_005'] + ny_model['PCT_SE_B17008_004'] 
ny_model['PCT_-2_ocp'] = ny_model['PCT_SE_B17008_012']
ny_model['PCT_-4_ocp'] = ny_model['PCT_SE_B17008_014'] + ny_model['PCT_SE_B17008_013']

ny_model_n = ny_model.drop(['PCT_SE_B17008_002', 'PCT_SE_B17008_003','PCT_SE_B17008_004', 'PCT_SE_B17008_005', 'PCT_SE_B17008_006','PCT_SE_B17008_007', 'PCT_SE_B17008_008', 'PCT_SE_B17008_009','PCT_SE_B17008_010', 'PCT_SE_B17008_011', 'PCT_SE_B17008_012','PCT_SE_B17008_013'], axis=1)

# print(ny_model_n.head())
# print(ny_model_n.shape)

# column_name_map = {
#                     'SE_A00002_002': 'Population Density',
#                     'PCT_SE_A03001_002': '% White population',
#                     'PCT_SE_B12001_004': ' % Bachelor or higher',
#                     'PCT_+4_ocp': '% Occupations - Highly likely WFH',
#                     'PCT_+2_ocp': '% Occupations - Very likely WFH',
#                     'PCT_-4_ocp': '% Occupations - Highly unlikely WFH',
#                     'PCT_SE_B14001_006': '% Households income $100,000 or More',
#                     'SE_A10036_001': 'Median House Value',
#                     'PCT_SE_A10030_002': '% No Vehicle household',
#                     'PCT_SE_A09005_008': 'WFH rate'
#                     }

# # Rename the columns using the dictionary
# ny_model.rename(columns=column_name_map, inplace=True)

# List of mobility variables you want to consider
mobility_vars = ['retail_and_recreation_percent_change_from_baseline', 
                 'grocery_and_pharmacy_percent_change_from_baseline', 
                 'parks_percent_change_from_baseline', 
                 'transit_stations_percent_change_from_baseline', 
                 'workplaces_percent_change_from_baseline', 
                 'residential_percent_change_from_baseline']

excluded_cols = mobility_vars + ['date', 'CountyName']
socioeconomic_vars = list(set(ny_model_n.columns) - set(excluded_cols))

# Creating a smaller DataFrame with only the columns you're interested in
df_selected = ny_model_n[socioeconomic_vars + mobility_vars]

# Compute pairwise correlation of columns, excluding NA/null values
correlation_matrix = df_selected.corr()

# Select only correlations with mobility variables
correlation_with_mobility = correlation_matrix.loc[socioeconomic_vars, mobility_vars]

# Only keep correlations with absolute value > 0.1
correlation_01 = correlation_with_mobility[correlation_with_mobility.abs() > 0.1].dropna(how='all')
correlation_02 = correlation_with_mobility[correlation_with_mobility.abs() > 0.2].dropna(how='all')

# Print the filtered correlation matrix, or analyse it further
# print(correlation_01)
print(correlation_02)

                   retail_and_recreation_percent_change_from_baseline  \
PCT_-1_ocp                                                 -0.351046    
SE_A18003_001                                              -0.223875    
PCT_+4_ocp                                                 -0.389027    
PCT_-2_ocp                                                  0.438056    
PCT_SE_B14001_006                                          -0.296889    
PCT_SE_B14001_002                                                NaN    
SE_A00001_001                                              -0.359071    
PCT_SE_B14001_003                                           0.345489    
SE_A10036_001                                              -0.407042    
PCT_SE_A02001_003                                          -0.350993    
PCT_SE_A10044_003                                           0.303906    
PCT_SE_B01001_003                                          -0.266881    
PCT_SE_B14001_005                                  

In [122]:
retail_recreation = ny_model[socioeconomic_vars + ['date', 'CountyName', 'retail_and_recreation_percent_change_from_baseline']]
retail_recreation = retail_recreation.dropna()

print(retail_recreation.head())
print(retail_recreation.shape)
# (59152, 38)

   PCT_-1_ocp  SE_A18003_001  PCT_+4_ocp  PCT_-2_ocp  PCT_SE_B14001_006  \
0        26.3           27.5       20.88        4.68              36.17   
1        26.3           27.5       20.88        4.68              36.17   
2        26.3           27.5       20.88        4.68              36.17   
3        26.3           27.5       20.88        4.68              36.17   
4        26.3           27.5       20.88        4.68              36.17   

   PCT_+1_ocp  PCT_SE_B14001_002  SE_A00001_001  PCT_SE_B14001_003  \
0        3.05               15.9         314679              17.85   
1        3.05               15.9         314679              17.85   
2        3.05               15.9         314679              17.85   
3        3.05               15.9         314679              17.85   
4        3.05               15.9         314679              17.85   

   SE_A10036_001  ...  PCT_SE_A10060_003  SE_A00002_002  PCT_SE_A10028_006  \
0       235200.0  ...              42.71       601

In [123]:
# Creating a smaller DataFrame with only the columns you're interested in
df_selected = retail_recreation[socioeconomic_vars + ['retail_and_recreation_percent_change_from_baseline']]

# Compute pairwise correlation of columns, excluding NA/null values
correlation_matrix = df_selected.corr()

# Select only correlations with mobility variables
correlation_with_mobility = correlation_matrix.loc[socioeconomic_vars, ['retail_and_recreation_percent_change_from_baseline']]

# Only keep correlations with absolute value > 0.1
correlation_01 = correlation_with_mobility[correlation_with_mobility.abs() > 0.1].dropna(how='all')
correlation_02 = correlation_with_mobility[correlation_with_mobility.abs() > 0.2].dropna(how='all')

# Print the filtered correlation matrix, or analyse it further
# print(correlation_01)
print(correlation_02)

# drop cols based on regression result
drop_cols = ['SE_A18003_001', 'PCT_SE_B14001_003', 'WFH rate', 'PCT_SE_A10060_003']


                   retail_and_recreation_percent_change_from_baseline
PCT_-1_ocp                                                 -0.351046 
SE_A18003_001                                              -0.223875 
PCT_+4_ocp                                                 -0.389027 
PCT_-2_ocp                                                  0.438056 
PCT_SE_B14001_006                                          -0.296889 
SE_A00001_001                                              -0.359071 
PCT_SE_B14001_003                                           0.345489 
SE_A10036_001                                              -0.407042 
PCT_SE_A02001_003                                          -0.350993 
PCT_SE_A10044_003                                           0.303906 
PCT_SE_B01001_003                                          -0.266881 
PCT_SE_B14001_005                                           0.344039 
SE_B14001_001                                              -0.378172 
PCT_SE_A03001_002   

In [124]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

selected_features = correlation_02.iloc[:, 0].index.tolist()

# Define the target variable
target = 'retail_and_recreation_percent_change_from_baseline'

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(retail_recreation[selected_features], retail_recreation[target], test_size=0.3, random_state=42)

# Add a constant to the independent value
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Conduct the multiple linear regression
model = sm.OLS(y_train, X_train).fit()

# Print the summary statistics of the regression model
print(model.summary())

# Predict the test set results
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate R-squared
r_sqr = model.rsquared

print(f'R square: {r_sqr}')
print(f'Root Mean Squared Error: {rmse}')

# all - 0.264


                                            OLS Regression Results                                            
Dep. Variable:     retail_and_recreation_percent_change_from_baseline   R-squared:                       0.263
Model:                                                            OLS   Adj. R-squared:                  0.262
Method:                                                 Least Squares   F-statistic:                     550.3
Date:                                                Thu, 06 Jul 2023   Prob (F-statistic):               0.00
Time:                                                        15:34:06   Log-Likelihood:            -1.6688e+05
No. Observations:                                               38587   AIC:                         3.338e+05
Df Residuals:                                                   38561   BIC:                         3.340e+05
Df Model:                                                          25                                         
C

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

selected_features = correlation_02.iloc[:, 0].index.tolist()

# Define the target variable
target = 'retail_and_recreation_percent_change_from_baseline'

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(retail_recreation[selected_features], retail_recreation[target], test_size=0.3, random_state=42)

# Create a Random Forest Regressor model and fit it to the training data
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Use the trained model to predict the target variable for the test data
predictions = model.predict(X_test)

# Evaluate the model performance by calculating the mean squared error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 400.96044571131455


## All - first time

Regression results for retail_and_recreation_percent_change_from_baseline:

                                            OLS Regression Results                                            
==============================================================================================================
Dep. Variable:     retail_and_recreation_percent_change_from_baseline   R-squared:                       0.265
Model:                                                            OLS   Adj. R-squared:                  0.265
Method:                                                 Least Squares   F-statistic:                     765.4
Date:                                                Thu, 06 Jul 2023   Prob (F-statistic):               0.00
Time:                                                        13:14:08   Log-Likelihood:            -2.3830e+05
No. Observations:                                               55125   AIC:                         4.767e+05
Df Residuals:                                                   55098   BIC:                         4.769e+05
Df Model:                                                          26                                         
Covariance Type:                                            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   12.2270     10.443      1.171      0.242      -8.242      32.696
% White population                       0.4557      0.026     17.840      0.000       0.406       0.506
SE_A18003_001                            0.1441      0.071      2.031      0.042       0.005       0.283
PCT_SE_B17008_011                       -0.9728      0.285     -3.412      0.001      -1.532      -0.414
PCT_SE_B17008_002                       -0.3857      0.121     -3.199      0.001      -0.622      -0.149
SE_A00001_001                         2.775e-05   3.16e-06      8.784      0.000    2.16e-05    3.39e-05
PCT_SE_B14001_003                        0.9633      0.117      8.226      0.000       0.734       1.193
PCT_SE_A02001_003                       -1.1150      0.117     -9.533      0.000      -1.344      -0.886
WFH rate                                 0.2128      0.097      2.202      0.028       0.023       0.402
PCT_SE_A10044_003                        0.0907      0.019      4.730      0.000       0.053       0.128
PCT_SE_B01001_003                       -0.5613      0.044    -12.689      0.000      -0.648      -0.475
**Population Density                      -0.0001   7.64e-05     -1.616      0.106      -0.000    2.63e-05
% No Vehicle household                   0.2817      0.090      3.124      0.002       0.105       0.458
PCT_SE_B17008_012                        0.7668      0.110      6.999      0.000       0.552       0.982
**PCT_SE_B14001_005                       -0.0016      0.113     -0.014      0.989      -0.224       0.221
SE_B14001_001                         -6.79e-05   8.94e-06     -7.596      0.000   -8.54e-05   -5.04e-05
PCT_SE_B17008_008                       -2.0411      0.253     -8.055      0.000      -2.538      -1.544
Median House Value                   -2.293e-05      3e-06     -7.633      0.000   -2.88e-05    -1.7e-05
**% Households income $100,000 or More    -0.0374      0.082     -0.458      0.647      -0.197       0.123
SE_A09003_001                            0.4845      0.050      9.784      0.000       0.387       0.582
**PCT_SE_B14001_004                        0.1076      0.114      0.944      0.345      -0.116       0.331
PCT_SE_B17008_013                       -0.9089      0.108     -8.404      0.000      -1.121      -0.697
PCT_SE_B17008_014                       -0.6875      0.140     -4.924      0.000      -0.961      -0.414
**PCT_SE_A10060_003                       -0.0411      0.038     -1.078      0.281      -0.116       0.034
PCT_SE_A10028_006                       -3.9054      0.946     -4.126      0.000      -5.760      -2.050
PCT_SE_B17008_003                       -0.8865      0.073    -12.213      0.000      -1.029      -0.744
 % Bachelor or higher                    0.3892      0.057      6.816      0.000       0.277       0.501
==============================================================================
Omnibus:                     4892.099   Durbin-Watson:                   0.432
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15996.277
Skew:                          -0.447   Prob(JB):                         0.00
Kurtosis:                       5.483   Cond. No.                     1.03e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.03e+08. This might indicate that there are
strong multicollinearity or other numerical problems.

================================================================================

Regression results for grocery_and_pharmacy_percent_change_from_baseline:

                                            OLS Regression Results                                           
=============================================================================================================
Dep. Variable:     grocery_and_pharmacy_percent_change_from_baseline   R-squared:                       0.249
Model:                                                           OLS   Adj. R-squared:                  0.249
Method:                                                Least Squares   F-statistic:                     756.6
Date:                                               Thu, 06 Jul 2023   Prob (F-statistic):               0.00
Time:                                                       13:14:08   Log-Likelihood:            -2.1834e+05
No. Observations:                                              52502   AIC:                         4.367e+05
Df Residuals:                                                  52478   BIC:                         4.369e+05
Df Model:                                                         23                                         
Covariance Type:                                           nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   98.5133      6.220     15.839      0.000      86.323     110.704
% White population                       0.0499      0.022      2.291      0.022       0.007       0.093
PCT_SE_B17008_011                       -7.3187      0.252    -29.080      0.000      -7.812      -6.825
PCT_SE_B17008_002                        0.2442      0.090      2.727      0.006       0.069       0.420
SE_A00001_001                         2.445e-05   2.54e-06      9.645      0.000    1.95e-05    2.94e-05
PCT_SE_B14001_003                       -0.5216      0.096     -5.411      0.000      -0.711      -0.333
PCT_SE_A02001_003                       -2.0889      0.091    -23.033      0.000      -2.267      -1.911
**WFH rate                                 0.0187      0.069      0.271      0.786      -0.117       0.154
PCT_SE_A10044_003                        0.4177      0.016     25.642      0.000       0.386       0.450
Population Density                      -0.0013   5.27e-05    -24.479      0.000      -0.001      -0.001
% No Vehicle household                   1.3710      0.063     21.605      0.000       1.247       1.495
PCT_SE_B17008_012                        1.9409      0.101     19.212      0.000       1.743       2.139
PCT_SE_B14001_005                       -0.4821      0.083     -5.815      0.000      -0.645      -0.320
SE_B14001_001                        -7.595e-05   7.18e-06    -10.585      0.000      -9e-05   -6.19e-05
Median House Value                    1.106e-05   2.46e-06      4.499      0.000    6.24e-06    1.59e-05
% Households income $100,000 or More    -0.2501      0.064     -3.899      0.000      -0.376      -0.124
SE_A09003_001                           -0.2788      0.041     -6.777      0.000      -0.359      -0.198
PCT_SE_B14001_004                        0.4096      0.089      4.621      0.000       0.236       0.583
PCT_SE_B17008_013                        0.4455      0.087      5.096      0.000       0.274       0.617
PCT_SE_B17008_014                        0.4475      0.115      3.894      0.000       0.222       0.673
PCT_SE_A10060_003                       -0.0875      0.028     -3.079      0.002      -0.143      -0.032
PCT_SE_A10028_006                      -11.4708      0.789    -14.537      0.000     -13.017      -9.924
PCT_SE_B17008_003                       -0.9453      0.065    -14.577      0.000      -1.072      -0.818
 % Bachelor or higher                    0.5460      0.045     12.184      0.000       0.458       0.634
==============================================================================
Omnibus:                     7160.502   Durbin-Watson:                   0.605
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            79408.718
Skew:                           0.260   Prob(JB):                         0.00
Kurtosis:                       9.002   Cond. No.                     7.24e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.24e+07. This might indicate that there are
strong multicollinearity or other numerical problems.

================================================================================

Regression results for parks_percent_change_from_baseline:

                                    OLS Regression Results                                    
==============================================================================================
Dep. Variable:     parks_percent_change_from_baseline   R-squared:                       0.190
Model:                                            OLS   Adj. R-squared:                  0.189
Method:                                 Least Squares   F-statistic:                     590.8
Date:                                Thu, 06 Jul 2023   Prob (F-statistic):               0.00
Time:                                        13:14:08   Log-Likelihood:            -1.1576e+05
No. Observations:                               20223   AIC:                         2.315e+05
Df Residuals:                                   20214   BIC:                         2.316e+05
Df Model:                                           8                                         
Covariance Type:                            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   -336.8469     11.204    -30.066      0.000    -358.807    -314.887
PCT_SE_B14001_002         -4.2652      0.262    -16.250      0.000      -4.780      -3.751
PCT_SE_B01001_003          1.3224      0.284      4.649      0.000       0.765       1.880
Population Density         0.0040      0.000      9.135      0.000       0.003       0.005
% No Vehicle household    -7.5181      0.621    -12.114      0.000      -8.735      -6.302
PCT_SE_B17008_009         31.4473      0.767     40.989      0.000      29.943      32.951
PCT_SE_B17008_013         17.7434      0.616     28.800      0.000      16.536      18.951
PCT_SE_A10060_003          3.8337      0.245     15.667      0.000       3.354       4.313
PCT_SE_A10028_006         86.2003      4.673     18.447      0.000      77.041      95.359
==============================================================================
Omnibus:                     6406.806   Durbin-Watson:                   0.535
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30131.311
Skew:                           1.474   Prob(JB):                         0.00
Kurtosis:                       8.203   Cond. No.                     4.56e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.56e+05. This might indicate that there are
strong multicollinearity or other numerical problems.

================================================================================

Regression results for transit_stations_percent_change_from_baseline:

                                          OLS Regression Results                                         
=========================================================================================================
Dep. Variable:     transit_stations_percent_change_from_baseline   R-squared:                       0.347
Model:                                                       OLS   Adj. R-squared:                  0.347
Method:                                            Least Squares   F-statistic:                     712.9
Date:                                           Thu, 06 Jul 2023   Prob (F-statistic):               0.00
Time:                                                   13:14:08   Log-Likelihood:            -1.3222e+05
No. Observations:                                          29476   AIC:                         2.645e+05
Df Residuals:                                              29453   BIC:                         2.647e+05
Df Model:                                                     22                                         
Covariance Type:                                       nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    677.1618     21.286     31.812      0.000     635.440     718.884
% White population         0.3073      0.040      7.633      0.000       0.228       0.386
PCT_SE_B17008_011          3.3975      0.705      4.820      0.000       2.016       4.779
PCT_SE_B17008_002         -1.6940      0.209     -8.086      0.000      -2.105      -1.283
SE_A00001_001           5.121e-05   4.82e-06     10.621      0.000    4.18e-05    6.07e-05
PCT_SE_B14001_003          1.8606      0.207      9.000      0.000       1.455       2.266
PCT_SE_A02001_003        -10.7275      0.342    -31.385      0.000     -11.397     -10.058
PCT_SE_A10044_003          0.8956      0.049     18.163      0.000       0.799       0.992
PCT_SE_B01001_003         -2.5168      0.101    -24.846      0.000      -2.715      -2.318
Population Density         0.0016      0.000     11.690      0.000       0.001       0.002
% No Vehicle household    -1.4152      0.183     -7.751      0.000      -1.773      -1.057
PCT_SE_B17008_012          4.6078      0.198     23.273      0.000       4.220       4.996
PCT_SE_B14001_005         -4.2286      0.208    -20.357      0.000      -4.636      -3.821
SE_B14001_001             -0.0001   1.37e-05    -10.494      0.000      -0.000      -0.000
PCT_SE_B17008_008        -16.5534      0.735    -22.516      0.000     -17.994     -15.112
Median House Value     -7.982e-05   4.39e-06    -18.161      0.000   -8.84e-05   -7.12e-05
SE_A09003_001             -0.6060      0.085     -7.152      0.000      -0.772      -0.440
PCT_SE_B14001_004         -1.2853      0.216     -5.950      0.000      -1.709      -0.862
PCT_SE_B17008_013         -4.5030      0.234    -19.204      0.000      -4.963      -4.043
PCT_SE_B17008_014         -7.5657      0.324    -23.323      0.000      -8.201      -6.930
PCT_SE_A10060_003          1.2796      0.072     17.858      0.000       1.139       1.420
PCT_SE_A10028_006         41.3740      2.543     16.268      0.000      36.389      46.359
 % Bachelor or higher      0.4785      0.086      5.563      0.000       0.310       0.647
==============================================================================
Omnibus:                     3218.043   Durbin-Watson:                   0.329
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10429.535
Skew:                           0.561   Prob(JB):                         0.00
Kurtosis:                       5.690   Cond. No.                     1.78e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.78e+08. This might indicate that there are
strong multicollinearity or other numerical problems.

================================================================================

Regression results for workplaces_percent_change_from_baseline:

                                       OLS Regression Results                                      
===================================================================================================
Dep. Variable:     workplaces_percent_change_from_baseline   R-squared:                       0.152
Model:                                                 OLS   Adj. R-squared:                  0.151
Method:                                      Least Squares   F-statistic:                     457.5
Date:                                     Thu, 06 Jul 2023   Prob (F-statistic):               0.00
Time:                                             13:14:09   Log-Likelihood:            -2.3710e+05
No. Observations:                                    58918   AIC:                         4.742e+05
Df Residuals:                                        58894   BIC:                         4.745e+05
Df Model:                                               23                                         
Covariance Type:                                 nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   35.0549      4.860      7.213      0.000      25.529      44.581
% White population                       0.2305      0.018     12.990      0.000       0.196       0.265
**PCT_SE_B17008_011                       -0.3171      0.167     -1.897      0.058      -0.645       0.011
PCT_SE_B17008_002                       -0.2084      0.069     -3.008      0.003      -0.344      -0.073
SE_A00001_001                         1.147e-05   2.11e-06      5.429      0.000    7.33e-06    1.56e-05
PCT_SE_B14001_003                       -0.2941      0.078     -3.787      0.000      -0.446      -0.142
PCT_SE_A02001_003                       -0.6902      0.064    -10.758      0.000      -0.816      -0.564
**WFH rate                                 0.0117      0.055      0.213      0.832      -0.096       0.119
PCT_SE_A10044_003                        0.1178      0.013      9.114      0.000       0.092       0.143
Population Density                       0.0002   4.26e-05      5.011      0.000       0.000       0.000
% No Vehicle household                  -0.5513      0.050    -11.071      0.000      -0.649      -0.454
**PCT_SE_B17008_012                        0.0594      0.070      0.845      0.398      -0.078       0.197
PCT_SE_B14001_005                       -0.8415      0.062    -13.636      0.000      -0.962      -0.721
SE_B14001_001                        -2.698e-05   5.98e-06     -4.515      0.000   -3.87e-05   -1.53e-05
**Median House Value                    3.599e-06   2.07e-06      1.741      0.082   -4.54e-07    7.65e-06
% Households income $100,000 or More    -0.1150      0.051     -2.275      0.023      -0.214      -0.016
SE_A09003_001                           -0.0729      0.032     -2.272      0.023      -0.136      -0.010
PCT_SE_B14001_004                       -0.4534      0.062     -7.359      0.000      -0.574      -0.333
PCT_SE_B17008_013                        0.3230      0.073      4.429      0.000       0.180       0.466
PCT_SE_B17008_014                       -0.5475      0.092     -5.940      0.000      -0.728      -0.367
PCT_SE_A10060_003                        0.4108      0.024     17.466      0.000       0.365       0.457
**PCT_SE_A10028_006                       -1.3395      0.671     -1.996      0.046      -2.655      -0.024
PCT_SE_B17008_003                       -0.2926      0.051     -5.774      0.000      -0.392      -0.193
 % Bachelor or higher                   -0.3656      0.036    -10.178      0.000      -0.436      -0.295
==============================================================================
Omnibus:                     6389.210   Durbin-Watson:                   0.884
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15251.215
Skew:                          -0.648   Prob(JB):                         0.00
Kurtosis:                       5.129   Cond. No.                     6.47e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.47e+07. This might indicate that there are
strong multicollinearity or other numerical problems.

================================================================================

Regression results for residential_percent_change_from_baseline:

                                       OLS Regression Results                                       
====================================================================================================
Dep. Variable:     residential_percent_change_from_baseline   R-squared:                       0.093
Model:                                                  OLS   Adj. R-squared:                  0.093
Method:                                       Least Squares   F-statistic:                     367.6
Date:                                      Thu, 06 Jul 2023   Prob (F-statistic):               0.00
Time:                                              13:14:09   Log-Likelihood:            -1.6822e+05
No. Observations:                                     53697   AIC:                         3.365e+05
Df Residuals:                                         53681   BIC:                         3.366e+05
Df Model:                                                15                                         
Covariance Type:                                  nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   -4.6845      1.297     -3.612      0.000      -7.226      -2.143
% White population                      -0.0585      0.005    -11.337      0.000      -0.069      -0.048
PCT_SE_B17008_002                        0.2864      0.028     10.414      0.000       0.233       0.340
**SE_A00001_001                        -3.085e-07   7.43e-07     -0.415      0.678   -1.76e-06    1.15e-06
**PCT_SE_B14001_003                        0.0247      0.028      0.873      0.383      -0.031       0.080
WFH rate                                -0.0285      0.020     -1.398      0.162      -0.068       0.011
Population Density                    8.989e-06   8.46e-06      1.062      0.288    -7.6e-06    2.56e-05
PCT_SE_B17008_012                        0.0188      0.028      0.670      0.503      -0.036       0.074
SE_B14001_001                         8.106e-07   2.13e-06      0.380      0.704   -3.37e-06    4.99e-06
Median House Value                   -3.069e-06   7.51e-07     -4.084      0.000   -4.54e-06    -1.6e-06
% Households income $100,000 or More     0.0001      0.017      0.008      0.993      -0.033       0.033
SE_A09003_001                            0.1379      0.011     12.611      0.000       0.116       0.159
PCT_SE_B14001_004                        0.0898      0.027      3.311      0.001       0.037       0.143
PCT_SE_B17008_013                        0.1799      0.022      8.336      0.000       0.138       0.222
PCT_SE_B17008_003                        0.1810      0.019      9.437      0.000       0.143       0.219
 % Bachelor or higher                    0.0408      0.013      3.177      0.001       0.016       0.066
==============================================================================
Omnibus:                     8424.520   Durbin-Watson:                   0.648
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16915.431
Skew:                           0.962   Prob(JB):                         0.00
Kurtosis:                       4.965   Cond. No.                     4.21e+07
==============================================================================

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

for mobility_var in mobility_vars:
    # Get the factors that have strong correlation with the current mobility variable
    factors = correlation_02[mobility_var].dropna().index.tolist()

    if factors:  # If there are factors that correlate strongly
        # Define the independent (X) and dependent (y) variables
        X = ny_model[factors]
        y = ny_model[mobility_var]

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Fit the Random Forest model
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate the mean squared error of the predictions
        mse = mean_squared_error(y_test, y_pred)

        # Print the results
        print(f'Mean Squared Error for {mobility_var}: {mse}')
        print("\n" + "="*80 + "\n")
    else:
        print(f'No strong correlations found for {mobility_var}.\n')

ValueError: Found input variables with inconsistent numbers of samples: [59152, 55125]